# 各種Seq2Seqモデルを試す

## simple_seq2seq

seq2seqの一番基本となる，LSTMによるencoder-decoderモデル

### 準備

In [1]:
import pandas as pd
import numpy as np
import janome
from janome.tokenizer import Tokenizer


from collections import Counter
from keras.preprocessing import text, sequence
from keras.utils import np_utils

from sklearn.model_selection import train_test_split
from keras.models import Model, load_model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, CuDNNLSTM
from keras.optimizers import Adam

C:\Users\akio\Anaconda3\envs\keras-gpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
t = Tokenizer()
def get_wakati(text, reverse=False, offset=False):
    """
    text: 文書(str) 
    reverse=Trueの場合は，文字の並びを反転させる．（seq2seqでは，入力系列は反転しなければならない！）
    offset=Trueの場合は，タイムステップ文
    """
    wakati_list = t.tokenize(text, wakati=True)
    if reverse == True:
        wakati_list = list(reversed(wakati_list))
    return " ".join(wakati_list)

In [3]:
def words_count(df, target_cols):
    total_word_count = Counter()
    for col in target_cols:
        for text in df[col]: 
            total_word_count += Counter(text.split())
    #print(total_word_count.keys())
    return len(total_word_count.keys())

In [4]:
def charlen_count(df, target_colname):
    df["tmp_wlist"] = df[target_colname].str.split()
    df[target_colname+"_charlen"] = df["tmp_wlist"].apply(lambda x: len(x))
    df = df.drop(["tmp_wlist"], axis=1)
    return df

In [5]:
def seq2seq_model(new_params=None):
    """
    最も基本的な4層LSTMによる，encoder-decoderモデル
    Ref: https://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf
    
    hyper_params: カスタム設定したハイパーパラメータ
    """
    params = {
        "seq_maxlen": 139,
        "num_words": 1592,
        "vec_len": 128,
        "learning_rate": 0.002,
        #"dropout_rate": 0.5,
        "loss_func": "categorical_crossentropy",
        "metrics": ["accuracy"]
    }
    if new_params != None:
        params.update(new_params)
        
    enc_inp = Input(shape=(params["seq_maxlen"], params["num_words"]))
    # decoder初期化用の内部状態を得る
    enc_out, state_h, state_c = CuDNNLSTM(params["vec_len"], return_sequences=True, return_state=True)(enc_inp)
    enc_states = [state_h, state_c]
    
    dec_out = CuDNNLSTM(params["vec_len"], return_sequences=True)(enc_out, initial_state=enc_states)
    seq_out = Dense(params["num_words"], activation="softmax")(dec_out)
    
    model = Model(inputs=enc_inp, outputs=seq_out)
    model.compile(loss=params["loss_func"], 
                  optimizer=Adam(lr=params["learning_rate"]),
                  metrics=params["metrics"]) # BLEUスコアで評価したいが，Kerasでは難しいので取り敢えず正解率で設定
    model.summary() # モデルの構成を表示
    return model

### 前処理

In [6]:
# データは各自用意する．
train_df = pd.read_csv("seq2seq_sample.csv")
train_df.head()

,__text,reaction
0,おはきゃーっと！,neglect
1,ねこますかわいいいいい,ありがと♡
2,のら！ちゃん！べりべりきゅーと！,neglect
3,おはきゃーっと！,neglect
4,🔥猫松🔥,猫松さんが燃やされていますね


In [7]:
train_df["wakati_enc_in"] = train_df["__text"].apply(lambda x: get_wakati(x, reverse=True))
train_df["wakati_dec_out"] = train_df["reaction"].apply(lambda x: get_wakati(x))
train_df.head()

,__text,reaction,wakati_enc_in,wakati_dec_out
0,おはきゃーっと！,neglect,！ っと きゃー は お,neglect
1,ねこますかわいいいいい,ありがと♡,いい いい わい か ます ねこ,ありがと ♡
2,のら！ちゃん！べりべりきゅーと！,neglect,！ と ー ゅ き べり べり ！ ちゃん ！ のら,neglect
3,おはきゃーっと！,neglect,！ っと きゃー は お,neglect
4,🔥猫松🔥,猫松さんが燃やされていますね,🔥 松 猫 🔥,猫 松 さん が 燃やさ れ て い ます ね


In [8]:
# 学習データの語彙数を計算する
target_cols = ["wakati_enc_in", "wakati_dec_out"]
num_words = words_count(train_df, target_cols)

In [9]:
for col in target_cols:
    train_df = charlen_count(train_df, target_colname=col)

In [10]:
# 系列長順に並べ替える，ミニバッチ学習で系列長の差を小さくするため．
train_df = train_df.sort_values(by=target_cols)

In [11]:
# テキストを数値ベクトル表現に変換する
# 1. 変換器の学習
tokenizer = text.Tokenizer(num_words=num_words)
inout_text_list = []
for col in target_cols:
    inout_text_list += train_df[col].tolist()
tokenizer.fit_on_texts(inout_text_list)
# 2. 最大の系列長を計算
inout_text_len = [len(text) for text in inout_text_list]
seq_maxlen = np.max(inout_text_len)
# 3. 数値ベクトルに変換
token_enc_in = tokenizer.texts_to_sequences(train_df["wakati_enc_in"].values)
token_dec_out = tokenizer.texts_to_sequences(train_df["wakati_dec_out"].values)
# 4. 系列長を揃える
token_enc_in = sequence.pad_sequences(token_enc_in, maxlen=seq_maxlen)
token_dec_out = sequence.pad_sequences(token_dec_out, maxlen=seq_maxlen)
# 5. one-hot形式に変換
onehot_enc_in = np_utils.to_categorical(token_enc_in, num_classes=num_words) 
onehot_dec_out = np_utils.to_categorical(token_dec_out, num_classes=num_words)

In [12]:
# 数値ベクトルに変換した特徴量の例
token_dec_out[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1])

0は文字列なし，1はEOS．それ以外は，単語が数値に変換されている
逆変換する場合は，1以上の要素を抽出して，

```
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))
```

で逆変換の辞書を作成して，系列を求めればよい

In [13]:
onehot_dec_out[0]

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [14]:
# 数値データに戻す方法
np.argmax(onehot_dec_out[0,138,:])

1

### 学習

#### Train-test split

In [15]:
params = {
    "seq_maxlen": seq_maxlen,
    "num_words": num_words,
    "vec_len": 128,
    "learning_rate": 0.002,
    #"dropout_rate": 0.5,
    "loss_func": "categorical_crossentropy",
    "metrics": ["accuracy"]
}

model = seq2seq_model(params)
# batchは論文を参考に設定．最適化はしていない
# epochsはある程度回さないと最適解に到達しない！
batch_size = 128
epochs = 90
history = model.fit(onehot_enc_in, onehot_dec_out, 
                    batch_size=batch_size, epochs=epochs, validation_split=0.1, 
                    shuffle="batch", # バッチデータ内でシャッフルする 
                    verbose=1)

Instructions for updating:
Use the retry module or similar alternatives.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 139, 1590)    0                                            
__________________________________________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)        [(None, 139, 128), ( 880640      input_1[0][0]                    
__________________________________________________________________________________________________
cu_dnnlstm_2 (CuDNNLSTM)        (None, 139, 128)     132096      cu_dnnlstm_1[0][0]               
                                                                 cu_dnnlstm_1[0][1]               
                                                                 cu_dnnlstm_1[0][2]               
____________________________________

Epoch 48/90
3878/3878 [==============================] - 20s 5ms/step - loss: 0.0541 - acc: 0.9886 - val_loss: 0.7663 - val_acc: 0.9361
Epoch 49/90
3878/3878 [==============================] - 20s 5ms/step - loss: 0.0516 - acc: 0.9886 - val_loss: 0.7837 - val_acc: 0.9361
Epoch 50/90
3878/3878 [==============================] - 21s 5ms/step - loss: 0.0513 - acc: 0.9886 - val_loss: 0.7657 - val_acc: 0.9361
Epoch 51/90
3878/3878 [==============================] - 20s 5ms/step - loss: 0.0517 - acc: 0.9887 - val_loss: 0.7649 - val_acc: 0.9361
Epoch 52/90
3878/3878 [==============================] - 20s 5ms/step - loss: 0.0516 - acc: 0.9886 - val_loss: 0.7472 - val_acc: 0.9361
Epoch 53/90
3878/3878 [==============================] - 20s 5ms/step - loss: 0.0503 - acc: 0.9886 - val_loss: 0.7702 - val_acc: 0.9361
Epoch 54/90
3878/3878 [==============================] - 20s 5ms/step - loss: 0.0505 - acc: 0.9886 - val_loss: 0.7483 - val_acc: 0.9361
Epoch 55/90
3878/3878 [=========================

### 推論

#### 準備

In [30]:
# 推定対称の入力系列を用意
example_testtext = "ねこますかわいいいいい" 
wakati_testtext= get_wakati(example_testtext, reverse=True)
# 数値ベクトルに変換　tokenizerは学習データの特徴量を作成したときのモデルを使用
token_testinput = tokenizer.texts_to_sequences(np.array([wakati_testtext]))
# 系列長を揃える
token_testinput = sequence.pad_sequences(token_testinput, maxlen=seq_maxlen)
# one-hot表現に変換
onehot_testinput = np_utils.to_categorical(token_testinput, num_classes=num_words)

#### 推論

In [31]:
onehot_test_output = model.predict(onehot_testinput, verbose=1)

1/1 [==============================] - 0s 14ms/step


In [32]:
onehot_test_output

array([[[9.9475431e-01, 1.7740836e-04, 1.1747601e-06, ...,
         1.1553026e-06, 1.3781611e-06, 1.3927983e-06],
        [9.9807692e-01, 2.5602206e-04, 3.1289243e-08, ...,
         3.0148879e-08, 4.7967795e-08, 4.1928526e-08],
        [9.9817407e-01, 2.8862883e-04, 1.5719570e-08, ...,
         1.5231048e-08, 2.5864795e-08, 2.1755719e-08],
        ...,
        [9.8426211e-01, 3.4647244e-03, 3.0715867e-08, ...,
         2.8256336e-08, 4.6093803e-08, 3.8952599e-08],
        [7.7308244e-01, 9.3649246e-02, 9.4877919e-08, ...,
         8.0581763e-08, 1.2930089e-07, 1.1510024e-07],
        [1.9019218e-01, 5.3921276e-01, 7.1187365e-08, ...,
         5.6246751e-08, 8.9453629e-08, 8.1284341e-08]]], dtype=float32)

In [33]:
# 推定したone-hotデータを数値ベクトルに変換する
token_testoutput = [np.argmax(seq_val) for seq_val in onehot_test_output[0]]

In [34]:
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))
reverse_word_map

{1: 'neglect',
 2: '！',
 3: '♡',
 4: 'ありがと',
 5: 'かわいい',
 6: 'お',
 7: 'て',
 8: '松',
 9: '猫',
 10: '🔥',
 11: 'ます',
 12: '草',
 13: 'い',
 14: 'ー',
 15: 'が',
 16: 'さん',
 17: 'おお',
 18: 'ね',
 19: 'れ',
 20: '？',
 21: '燃やさ',
 22: 'の',
 23: 'べり',
 24: 'おい',
 25: 'いい',
 26: 'た',
 27: 'のら',
 28: 'じゃ',
 29: 'で',
 30: 'き',
 31: 'と',
 32: '～',
 33: 'あら',
 34: 'し',
 35: '＾',
 36: 'ない',
 37: 'ゅ',
 38: 'つ',
 39: 'う',
 40: 'は',
 41: 'っと',
 42: 'また',
 43: 'こく',
 44: 'きゃー',
 45: 'やっ',
 46: 'わかる',
 47: 'か',
 48: 'ひじき',
 49: 'いか',
 50: 'わかり',
 51: '今度',
 52: '会い',
 53: 'ましょ',
 54: 'ちゃん',
 55: '・',
 56: 'に',
 57: 'お前',
 58: 'ｗ',
 59: 'ｗｗｗｗ',
 60: 'も',
 61: 'な',
 62: 'おまえ',
 63: '🔥🔥',
 64: '🔥🔥🔥',
 65: 'おじ',
 66: 'お疲れ様',
 67: 'だ',
 68: 'おおお',
 69: 'でし',
 70: '…',
 71: 'を',
 72: 'よ',
 73: '皆さん',
 74: '，',
 75: 'ゃっち',
 76: 'み',
 77: 'かわい',
 78: 'ｗｗｗｗｗ',
 79: 'ん',
 80: 'やつ',
 81: 'てる',
 82: 'から',
 83: 'マン',
 84: 'です',
 85: 'おじさん',
 86: '、',
 87: 'ま',
 88: 'ｗｗｗｗｗｗ',
 89: 'け',
 90: 'おる',
 91: 'さ',
 92: 'つかれ',
 93:

In [35]:
token_testoutput = [val for val in token_testoutput if val > 0]
pred_seq = [reverse_word_map[key_val] for key_val in token_testoutput]

In [36]:
pred_seq

['neglect']